# Overview

This notebook assumes you have already collected and scored MOC tweets. It creates a dataset for use in R to analyze the patterns of polarization over time. You will do some parsing on an AWS server and some locally before ultimately making a CSV file that you can open and analyze in R.

LH Note: on my computer, Git stuff and data live in different places, so you'll see notes about moving files or changing directories. I haven't figured out a good way to keep files in both places or to mirror or sync or whatever. So, for now, paths are hard-coded or there's a note about where to find a file.

# Get Data

2016 election data is on an AWS server under ```/data/purpletag```. 

To login: 

```ssh -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org```

The data is large (> 4GB), so best to run Juypter notebooks to parse on the server. Then CSV files can be used locally.

You can run a notebook on the server and use your local browser with these two commands:

* ```ssh -L 8080:localhost:8888 -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org```
* ```nohup jupyter notebook --no-browser > log.txt 2>&1 &```

Then access ```http://localhost:8080``` in your browser.

## On server: Parsing from ```scores``` files to CSV

This section assumes you have already run purpletag's ```collect``` and ```score``` functions and gotten the Twitter data that you want in JSON format and parsed that data into score files.

In [ ]:
# Based on https://stackoverflow.com/questions/26415906/read-multiple-txt-files-into-pandas-dataframe-with-filename-as-column-header
import pandas as pd
import os
import glob

# Step 1: get a list of all score files in target directory
my_dir = "/data/purpletag/scores/"
filelist = []
filesList = []
os.chdir( my_dir )

# Step 2: Build up list of files:
for files in glob.glob("*.1.moc.scores"): # using 1-day purpletag MOC scores
    fileName, fileExtension = os.path.splitext(files)
    filesList.append(files) #filename with extension

In [ ]:
# Step 3: Build up DataFrame:
# Based on https://stackoverflow.com/questions/35717706/python-how-to-turn-a-dictionary-of-dataframes-into-one-big-dataframe-with-colum
d = {} # dictionary to hold multiple dfs

for filename in filesList:
    df1 = pd.read_csv(filename, header=None, sep=' ', index_col=0)
    d[filename[:-13]] = df1
    
df = pd.concat(d, axis=1)
df.columns = df.columns.droplevel(-1) 

df.to_pickle('/data/purpletag/mocs_by_date.pkl')

Move the file from the AWS server to local if you want to work locally:

```scp -i ~/.ssh/carolgrrr.pem ubuntu@purpletag.casmlab.org:/data/purpletag/mocs_by_date.pkl ~/Documents/git/casmlab/purpletag/files/```

## Locally: Prepping for stats

We now have a pickled dataframe of the form handleXdate. We need to keep data only from Labor Day to Election Day and get weekly averages.

In [146]:
import pandas as pd

df = pd.read_pickle('data-files/mocs_by_date.pkl')
df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-10-30,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-11-06,2016-11-07,2016-11-08
austinscottga08,NaN,2.40933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
benniegthompson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.56379,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bettymccollum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-90.6103,-69.5725,-50.1429,NaN,-33.4513,NaN,NaN,NaN
billpascrell,NaN,NaN,-1.168750,-0.3731,-0.972477,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-18.7023,NaN,NaN,NaN,NaN,NaN,NaN
boblatta,NaN,NaN,0.965309,NaN,NaN,0.612764,NaN,1.56897,NaN,-0.031978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
week1_dates = ['2016-09-06','2016-09-07','2016-09-08','2016-09-09','2016-09-10','2016-09-11','2016-09-12']
week2_dates = ['2016-09-13','2016-09-14','2016-09-15','2016-09-16','2016-09-17','2016-09-18','2016-09-19']
week3_dates = ['2016-09-20','2016-09-21','2016-09-22','2016-09-23','2016-09-24','2016-09-25','2016-09-26']
week4_dates = ['2016-09-27','2016-09-28','2016-09-29','2016-09-30','2016-10-01','2016-10-02','2016-10-03']
week5_dates = ['2016-10-04','2016-10-05','2016-10-06','2016-10-07','2016-10-08','2016-10-09','2016-10-10']
week6_dates = ['2016-10-11','2016-10-12','2016-10-13','2016-10-14','2016-10-15','2016-10-16','2016-10-17']
week7_dates = ['2016-10-18','2016-10-19','2016-10-20','2016-10-21','2016-10-22','2016-10-23','2016-10-24']
week8_dates = ['2016-10-25','2016-10-26','2016-10-27','2016-10-28','2016-10-29','2016-10-30','2016-10-31']
week9_dates = ['2016-11-01','2016-11-02','2016-11-03','2016-11-04','2016-11-05','2016-11-06','2016-11-07']

df['week1'] = df[week1_dates].mean(axis=1)
df['week2'] = df[week2_dates].mean(axis=1)
df['week3'] = df[week3_dates].mean(axis=1)
df['week4'] = df[week4_dates].mean(axis=1)
df['week5'] = df[week5_dates].mean(axis=1)
df['week6'] = df[week6_dates].mean(axis=1)
df['week7'] = df[week7_dates].mean(axis=1)
df['week8'] = df[week8_dates].mean(axis=1)
df['week9'] = df[week9_dates].mean(axis=1)

df.head()

,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-14,2015-11-15,2015-11-16,2015-11-17,2015-11-18,2015-11-19,...,2016-11-08,week1,week2,week3,week4,week5,week6,week7,week8,week9
austinscottga08,NaN,2.40933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.932035,NaN,1.064290,14.208100,6.042600,1.561610,NaN,NaN
benniegthompson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.56379,NaN,NaN,...,NaN,NaN,-1.211380,NaN,NaN,NaN,-1.823010,NaN,NaN,NaN
bettymccollum04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-120.131512,-467.500940,-89.054550,-86.397562,-29.160648,-35.474095,-5.049413,-33.368257,-60.94425
billpascrell,NaN,NaN,-1.168750,-0.3731,-0.972477,NaN,NaN,NaN,NaN,NaN,...,NaN,-2.871430,-2.577447,-2.353348,-3.437217,NaN,-1.026550,NaN,-1.333330,-18.70230
boblatta,NaN,NaN,0.965309,NaN,NaN,0.612764,NaN,1.56897,NaN,-0.031978,...,NaN,1.468710,13.197433,NaN,NaN,NaN,0.974138,NaN,NaN,NaN


In [148]:
weekly_df = df[['week1','week2','week3','week4','week5','week6','week7','week8','week9']]
weekly_df

,week1,week2,week3,week4,week5,week6,week7,week8,week9
austinscottga08,NaN,4.932035,NaN,1.064290,14.208100,6.042600,1.561610,NaN,NaN
benniegthompson,NaN,-1.211380,NaN,NaN,NaN,-1.823010,NaN,NaN,NaN
bettymccollum04,-120.131512,-467.500940,-89.054550,-86.397562,-29.160648,-35.474095,-5.049413,-33.368257,-60.944250
billpascrell,-2.871430,-2.577447,-2.353348,-3.437217,NaN,-1.026550,NaN,-1.333330,-18.702300
boblatta,1.468710,13.197433,NaN,NaN,NaN,0.974138,NaN,NaN,NaN
bradsherman,NaN,-1.941180,NaN,-10.763900,NaN,NaN,NaN,NaN,-0.748092
call_me_dutch,-2.754159,-5.169143,-3.129013,-1.573270,-2.967365,-2.715476,-49.760298,-1.576630,-6.732820
candicemiller,1.483315,NaN,-0.137418,-0.264974,NaN,1.086960,NaN,NaN,NaN
cathymcmorris,31.845395,4.472548,29.771150,39.539832,1.000000,17.632060,4.519305,53.018459,38.064545
cbrangel,-59.337715,-143.615365,-80.236928,-66.473658,-19.118496,-35.237185,-8.603265,-4.536950,-27.031432


In [164]:
import pandas as pd
import yaml

# get the data from Govtrack
with open('/Users/libbyh/Dropbox/CASM/SMCE/Shared Social Media and Civic Engagement/Data/purpletag/legislators-social-media.yaml', 'r') as f:
    df_social = pd.io.json.json_normalize(yaml.load(f))

with open('/Users/libbyh/Dropbox/CASM/SMCE/Shared Social Media and Civic Engagement/Data/purpletag/legislators-current.yaml', 'r') as f:
    df_current = pd.io.json.json_normalize(yaml.load(f))

print(len(weekly_df))
# merge everything into one data frame with one row per MOC
df_meta = pd.merge(df_current, df_social, on="id.govtrack")
df_meta["handle"] = df_meta["social.twitter"].str.lower()
weekly_df["handle"] = weekly_df.index.str.lower()

print(len(df_meta))

df_merged = pd.merge(df_meta, weekly_df, left_on="handle", right_index=True)

print(len(df_merged))

#cols_to_keep = ['id.govtrack','social.twitter','name.official_full','bio.gender','terms','week1','week2','week3','week4','week5','week6','week7','week8','week9']

df_merged = df_merged[['id.govtrack','social.twitter','name.official_full','bio.gender','terms','week1','week2','week3','week4','week5','week6','week7','week8','week9']]


511
529
444


Not sure why we have only 444 matches, but it's better than 12.

In [165]:
df1 = pd.concat([df_merged.drop(['terms'], axis=1), df_merged['terms'].apply(pd.Series)], axis=1)
df2 = pd.concat([df1.drop([0], axis=1), df1[0].apply(pd.Series)], axis=1)

keep_df = df2[['id.govtrack','social.twitter','name.official_full','bio.gender','type','party','week1','week2','week3','week4','week5','week6','week7','week8','week9']]
keep_df

,id.govtrack,social.twitter,name.official_full,bio.gender,type,party,week1,week2,week3,week4,week5,week6,week7,week8,week9
0,400050,SenSherrodBrown,Sherrod Brown,M,rep,Democrat,-0.222430,-1.141480,-2.894056,-28.188720,-0.587860,1.175650,-19.819673,-2.136967,-0.367748
1,300018,SenatorCantwell,Maria Cantwell,F,rep,Democrat,-118.229375,-12.501053,-5.904561,-39.350840,-11.031340,-20.399800,-33.510324,-18.325139,-28.551134
2,400064,SenatorCardin,Benjamin L. Cardin,M,rep,Democrat,-25.636298,-8.955374,-10.335548,-28.631307,0.866007,-25.627037,-16.434565,-21.518943,-17.753120
3,300019,SenatorCarper,Thomas R. Carper,M,rep,Democrat,-155.209127,-0.681740,-8.735320,-12.539793,-14.515218,-0.307598,-6.753922,-1.830320,-6.501631
4,412246,SenBobCasey,"Robert P. Casey, Jr.",M,sen,Democrat,-36.255503,-6.993170,-1.272941,-28.430050,-0.082187,-5.102067,-16.612726,-12.302640,-77.053400
5,412248,SenBobCorker,Bob Corker,M,sen,Republican,1.884995,8.134470,3.616570,7.727762,NaN,1.819670,NaN,1.748260,NaN
6,300043,SenFeinstein,Dianne Feinstein,F,sen,Democrat,-63.976412,-759.904700,-38.750658,-55.663120,-7.274102,-6.091820,-29.935897,NaN,-161.321474
7,300052,SenOrrinHatch,Orrin G. Hatch,M,sen,Republican,78.846271,155.737157,147.056983,155.817800,20.409763,66.542193,16.915500,52.884700,2.214622
9,412243,McCaskillOffice,Claire McCaskill,F,sen,Democrat,-4.946375,-728.367104,-52.026723,-3.490048,-0.763953,-10.002037,-0.857143,-2.249950,-0.748092
10,400272,SenatorMenendez,Robert Menendez,M,rep,Democrat,-36.628585,-9.758632,-0.829845,-29.610837,-2.319673,-19.566032,-16.767273,-6.268110,-242.438693


In [170]:
# melt it so each row is a person x week
df_long = pd.melt(keep_df, id_vars=['id.govtrack','social.twitter','name.official_full','bio.gender','party','type'],
                value_vars=['week1','week2','week3','week4','week5','week6','week7','week8','week9'],
                var_name='week', value_name='avg_score')
df_long['week'] = df_long['week'].str[-1:]

df_long.rename(columns = {'type':'chamber', 'social.twitter': 'handle', 'name.official_full': 'name', 'bio.gender': 'gender'}, inplace = True)

df_long.head()

,id.govtrack,handle,name,gender,party,chamber,week,avg_score
0,400050,SenSherrodBrown,Sherrod Brown,M,Democrat,rep,1,-0.222430
1,300018,SenatorCantwell,Maria Cantwell,F,Democrat,rep,1,-118.229375
2,400064,SenatorCardin,Benjamin L. Cardin,M,Democrat,rep,1,-25.636298
3,300019,SenatorCarper,Thomas R. Carper,M,Democrat,rep,1,-155.209127
4,412246,SenBobCasey,"Robert P. Casey, Jr.",M,Democrat,sen,1,-36.255503


In [175]:
# make sure we have just two parties
df_long.party.unique()

array(['Democrat', 'Republican'], dtype=object)

In [176]:
# get an absolute value of the polar score
df_long['abs'] = df_long['avg_score'].abs()

In [177]:
df_long.to_csv('data-files/weekly_averages_long.csv')

# Now move to R for analysis

Run ```~/Documents/git/casmlab/purpletag/2016_election.R```

That R script sends its output to ```2016_election_results.txt```

In [180]:
results = open('data-files/2016_election_results.txt', 'r')
print(results.read())


> # for pretty regression tables
> # http://stackoverflow.com/questions/30195718/stargazer-save-to-file-dont-show-in-console
> mod_stargazer <- functi .... [TRUNCATED] 

> df <- read.csv('weekly_averages_long.csv', header = TRUE, sep = ",", quote = "\"",
+                dec = ".", fill = TRUE, comment.char = "")

> summary(df)
       X           id.govtrack                 handle                 name      gender  
 Min.   :   0.0   Min.   :300002   AustinScottGA08:   9   Adam B. Schiff:   9   F: 792  
 1st Qu.: 998.8   1st Qu.:400326   BennieGThompson:   9   Adam Kinzinger:   9   M:3204  
 Median :1997.5   Median :412292   BettyMcCollum04:   9   Adam Smith    :   9           
 Mean   :1997.5   Mean   :401868   BillPascrell   :   9   Adrian Smith  :   9           
 3rd Qu.:2996.2   3rd Qu.:412533   BobLatta       :   9   Al Franken    :   9           
 Max.   :3995.0   Max.   :412674   BradSherman    :   9   Al Green      :   9           
                                   (Other)    

Based on the outlier-excluded linear mixed-effects models, it makes sense to remove RepThompson. The pattern stays the same even with RepThompson in the set though: negative effect for republican and week, positive effect for their interaction. ```lmm5``` is the model-of-best-fit. 

## Changing the way we score hashtags

What if we score tags for the 63-day period and then score MOCS?

Run the following (on the server) to get new scores:

* purpletag parse -t 63 -d 200
* purpletag score
* purpletag score --counts --score-mocs

That first command took a week because the code starts with today and works backwards 200 days, one day at a time. Each day takes over an hour. See Issue #18 about options for changing this behavior.

With the new tag measures, can start the process over. Start at "On Server: Parsing scores to CSV" with a new file name.